<a href="https://colab.research.google.com/github/jangje0908/sesac/blob/main/Chapter4_A_to_Z_%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B5_Penguin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4. A to Z 빅데이터 분석 실습  

## 2.실습 A to Z – Penguin

### (1) 문제

#### Penguin Dataset
- https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv

2014년 남극 펭귄 공동체 내 생태학적 성적 이형성과 환경적 변동성에 관한 연구 데이터이며, 다양한 종의 펭귄들의 서식지와 주요 신체 크기와 무게에 대한 7가지 정도의 데잍터로 이루어져있다.
Palmer's penguin이라는 데이터 세트로도 알려져있으며 시각화 패키지 중 하나인 seaborn 패키지에 예제 데이터 세트로 존재한다.
이를 기반으로 펭귄의 무게를 예측하는 회귀 분석 진행해보자.

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 637, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 637 (delta 136), reused 184 (delta 109), pack-reused 363
Receiving objects: 100% (637/637), 10.76 MiB | 17.06 MiB/s, done.
Resolving deltas: 100% (321/321), done.


- penguin_y_train.csv: 펭귄의 무게 데이터(학습용)
- penguin_X_train.csv, penguin_X_test.csv: 펭귄의 속성(학습용 및 평가용)

- species: 펭귄 종
- island: 섬(서식지)
- sex: 성별
- bill_length_mm: 부리의 길이
- bill_depth_mm: 부리의 높이
- flipper_length_mm: 팔 길이
- body_mass_g: 체중 -> 종속 변수

### (2) 사전준비

#### 01

In [2]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_test.csv")
X_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_train.csv")
y_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)


####02

In [3]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')   # 경고 글귀 무시

#### 03

In [4]:
X_test = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_test.csv")
X_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_train.csv")
y_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_y_train.csv")

### (3) 데이터 전처리

#### 01

In [5]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            240 non-null    object 
 1   island             240 non-null    object 
 2   sex                232 non-null    object 
 3   bill_length_mm     238 non-null    float64
 4   bill_depth_mm      238 non-null    float64
 5   flipper_length_mm  238 non-null    float64
dtypes: float64(3), object(3)
memory usage: 11.4+ KB
None


- shape는 (240, 6)로 행 240개, 열 6개
- 6개 변수를 독립 변수로 둠


#### 02

In [6]:
train =  pd.concat([X_train, y_train], axis=1)
print(train.loc[(train.sex.isna()) | (train.bill_length_mm.isna()) | (train.bill_depth_mm.isna()) | (train.flipper_length_mm.isna()) | (train.body_mass_g.isna())])

    species     island  sex  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0    Adelie  Torgersen  NaN            42.0           20.2              190.0   
6    Gentoo     Biscoe  NaN            44.5           14.3              216.0   
43   Gentoo     Biscoe  NaN             NaN            NaN                NaN   
66   Adelie  Torgersen  NaN            37.8           17.3              180.0   
88   Gentoo     Biscoe  NaN            47.3           13.8              216.0   
89   Adelie  Torgersen  NaN            37.8           17.1              186.0   
110  Gentoo     Biscoe  NaN            44.5           15.7              217.0   
229  Adelie  Torgersen  NaN             NaN            NaN                NaN   

     body_mass_g  
0         4250.0  
6         4100.0  
43           NaN  
66        3700.0  
88        4725.0  
89        3300.0  
110       4875.0  
229          NaN  


- isna 함수를 사용하여 결측치가 존재하는 index를 찾아 loc 함수를 적용하면 어떤 row에 결측치가 있는지 확인 가능
- 여기서 중요한 것은 학습에 활용할 y_train 데이터를 포함하여 결측치 확인을 해야함
- concat 함수를 통해 두 데이터프레임 붙임. axis=1 조건으로 두 데이터프레임이 좌우로 붙음

- 결측치가 각기 다른 행에서 발생하지 않아 8개의 행만 처리

#### 03

In [7]:
train = train.dropna()
train.reset_index(drop=True, inplace=True)

- dropna함수를 통해 결측치 제거
- resset_index를 통해 인덱스를 초기화하여 추후 인덱스로 인한 혼란 방지

#### 04


In [8]:
X_train = train[['species','island', 'sex','bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y_train = train[['body_mass_g']]

- 결측치 처리 완료했으므로 붙였던 학습용 데이터를 다시 독립변수와 종속변수로 분리

#### 05

In [9]:
print(X_train.describe())

       bill_length_mm  bill_depth_mm  flipper_length_mm
count      232.000000     232.000000         232.000000
mean        43.990948      17.226293         200.681034
std          5.509760       1.964677          14.064231
min         32.100000      13.200000         172.000000
25%         39.200000      15.700000         190.000000
50%         44.950000      17.350000         197.000000
75%         48.775000      18.725000         212.250000
max         58.000000      21.500000         231.000000


- 분석에 큰 영향을 끼칠만한 이상치 없음
- 각 수치형 변수의 범위가 상이하기 때문에 추후 스케일링이 필요하다는 것 기억

#### 06

In [10]:
COL_DEL = []
COL_NUM = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']
COL_CAT = ['species','island', 'sex']
COL_Y = ['body_mass_g']

- 6개의 독립변수 중 3개의 수치형 변수와 3개의 범주형 변수 존재
- 수치형 변수는 모델링을 진행하면서 스케일링 진행, 범주형 변수는 레이블 인코딩 필요
- 스케일링은 평균과 분산에 영향을 줄 수 있기 때문에 학습데이터만으로 한정하여 진행해야 하지만, 레이블 인코딩은 그렇지 않으므로 현재 단계에서 모든 데이터를 포함하여 레이블 인코딩 진헹

#### 07

In [11]:
X = pd.concat([X_train, X_test])

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')  # handle_unknown: fit을 한 뒤에 transform을 진행할 때 처음 보는 범주가 등장했을 때 어떻게 처리할 것인지 결정하는 옵션
ohe.fit(X[COL_CAT])

X_train_res = ohe.transform(X_train[COL_CAT])
X_test_res = ohe.transform(X_test[COL_CAT])

#### 08

In [12]:
print(X_train_res)

  (0, 2)	1.0
  (0, 3)	1.0
  (0, 6)	1.0
  (1, 0)	1.0
  (1, 5)	1.0
  (1, 7)	1.0
  (2, 1)	1.0
  (2, 4)	1.0
  (2, 7)	1.0
  (3, 2)	1.0
  (3, 3)	1.0
  (3, 7)	1.0
  (4, 0)	1.0
  (4, 4)	1.0
  (4, 6)	1.0
  (5, 2)	1.0
  (5, 3)	1.0
  (5, 7)	1.0
  (6, 2)	1.0
  (6, 3)	1.0
  (6, 6)	1.0
  (7, 0)	1.0
  (7, 5)	1.0
  (7, 6)	1.0
  (8, 2)	1.0
  :	:
  (223, 6)	1.0
  (224, 2)	1.0
  (224, 3)	1.0
  (224, 7)	1.0
  (225, 0)	1.0
  (225, 4)	1.0
  (225, 7)	1.0
  (226, 1)	1.0
  (226, 4)	1.0
  (226, 7)	1.0
  (227, 1)	1.0
  (227, 4)	1.0
  (227, 6)	1.0
  (228, 2)	1.0
  (228, 3)	1.0
  (228, 7)	1.0
  (229, 0)	1.0
  (229, 5)	1.0
  (229, 6)	1.0
  (230, 1)	1.0
  (230, 4)	1.0
  (230, 6)	1.0
  (231, 2)	1.0
  (231, 3)	1.0
  (231, 6)	1.0


- transform된 데이터는 numpy arrayfh aksemfdjwls epdlxj

#### 09. 데이터프레임으로 변경

In [13]:
X_train_ohe = pd.DataFrame(X_train_res.todense(), columns = ohe.get_feature_names_out())
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns = ohe.get_feature_names_out())
print(X_train_ohe)

X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis=1)
X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis=1)

     species_Adelie  species_Chinstrap  species_Gentoo  island_Biscoe  \
0               0.0                0.0             1.0            1.0   
1               1.0                0.0             0.0            0.0   
2               0.0                1.0             0.0            0.0   
3               0.0                0.0             1.0            1.0   
4               1.0                0.0             0.0            0.0   
..              ...                ...             ...            ...   
227             0.0                1.0             0.0            0.0   
228             0.0                0.0             1.0            1.0   
229             1.0                0.0             0.0            0.0   
230             0.0                1.0             0.0            0.0   
231             0.0                0.0             1.0            1.0   

     island_Dream  island_Torgersen  sex_FEMALE  sex_MALE  
0             0.0               0.0         1.0       0.0  
1  

- todense 함수를 붙이고 ohedml get_feature_names 함수를 사용하여 변환된 범주형 변수를 만듦

### (4) 데이터 모형 구축

#### 01

In [14]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train_fin, y_train, test_size=0.3)

- 회귀 모형을 만들기 위해 학습용 데이터와 검증용 데이터 분할 진행

#### 02. 스케일링

In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM]=scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM]=scaler.transform(X_val[COL_NUM])
X_test_fin[COL_NUM]=scaler.transform(X_test_fin[COL_NUM])

sklearn패키지의 preprocessing 모듈에서 MinMaxScaler 함수를 import하여 스케일링을 진행

#### 03. 선형 회귀 모형 택하여 회귀분석 학습

In [16]:
from sklearn.linear_model import LinearRegression

modelLR = LinearRegression()
modelLR.fit(X_tr, y_tr)

y_val_pred = modelLR.predict(X_val)
print(y_val_pred)

[[3437.88915585]
 [3397.84090741]
 [3910.68936556]
 [5500.84347732]
 [3456.69842561]
 [3527.10442023]
 [4837.65141043]
 [4642.31972631]
 [5500.52759572]
 [5475.63453026]
 [3337.4255922 ]
 [3463.73008979]
 [4294.4383231 ]
 [5429.90049476]
 [3514.89330663]
 [5427.43069313]
 [4261.4137395 ]
 [3447.73477918]
 [4119.08244958]
 [5560.32237635]
 [4170.0342964 ]
 [4037.20619444]
 [3932.55688681]
 [3996.46732929]
 [3200.73412873]
 [5299.8589371 ]
 [5429.36266056]
 [3366.24738451]
 [4655.58947582]
 [3587.18387364]
 [3514.83380708]
 [3823.18030722]
 [4123.28731268]
 [3424.3130312 ]
 [3926.58556588]
 [3953.61894838]
 [3821.23301464]
 [5496.69573076]
 [3452.95346639]
 [4140.15337157]
 [3511.66779991]
 [4301.80205511]
 [3481.81170669]
 [3568.36235011]
 [5526.5139354 ]
 [5256.12217245]
 [4922.02251384]
 [5394.0652963 ]
 [3298.67582584]
 [4773.05835058]
 [4704.1736369 ]
 [4158.71915045]
 [3956.54065455]
 [3285.85291442]
 [3339.99992023]
 [3348.16997299]
 [5701.65518213]
 [4030.08848559]
 [4394.8665814

#### 04

In [21]:
print(modelLR.intercept_)   # intercept_: 절편 구하기

coef = pd.Series(data=modelLR.coef_[0], index= X_train_fin.columns)   # coef_: 기울기 구하기
print(coef.sort_values())

[3346.41145462]
species_Chinstrap   -454.193719
species_Adelie      -266.375090
sex_FEMALE          -208.799999
island_Dream         -17.520185
island_Biscoe         -6.593951
island_Torgersen      24.114136
sex_MALE             208.799999
bill_depth_mm        414.218132
bill_length_mm       441.625427
species_Gentoo       720.568810
flipper_length_mm    919.380534
dtype: float64



- 펭귄의 종이 'Chinstrap'이나 'Adelie'일 경우 무게에 음의 영향을 주고 있으며, 'Gentoo' 종일때 더 무렵다는 것을 알 수 있다.
- 성별의 경우 암컷이 수컷보다 작게 나오는 것으로 보이며 부리 의 크기는 클수록,  특히 날개의 길이는 무게에도 아주 큰 영향을 준다고 볼 수 있다.

### (5) 데이터 모형 평가

####01

In [18]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_val, y_val_pred)
rmse = mean_squared_error(y_val, y_val_pred, squared=False)

print('MSE : {0:.3f} , RMSE : {1:.3F} '.format(mse , rmse))

MSE : 89201.648 , RMSE : 298.666 


- RMSE는 MSE에 제곱근을 씌운 형태이며, MSE와 거의 동일한 방식이기 때문에 mean_squared_error 함수만 import하여 계산 할 수 있음

#### 02

In [19]:
y_pred = modelLR.predict(X_test_fin)
print(y_pred)

[[4130.13964058]
 [5335.85336797]
 [4025.97051342]
 [3448.16337405]
 [4679.83373078]
 [4040.01680929]
 [3397.03372557]
 [3654.90914861]
 [3542.15405783]
 [3504.47540484]
 [3389.74481827]
 [5489.82243791]
 [3536.8423027 ]
 [4072.614535  ]
 [3834.80655606]
 [4106.13830316]
 [4369.2461016 ]
 [3516.51911396]
 [3612.78904312]
 [3385.1089921 ]
 [5335.21902154]
 [4187.52733977]
 [5845.47311468]
 [4690.19063454]
 [5382.02958653]
 [5501.69891527]
 [5506.16512667]
 [4217.36259172]
 [3430.27704115]
 [4042.12333429]
 [4170.13727336]
 [3422.84172854]
 [3544.46548812]
 [4630.71259659]
 [3363.63217714]
 [3493.12379609]
 [3234.30471859]
 [3412.9640205 ]
 [3920.38785234]
 [4954.58072883]
 [3495.44733089]
 [5590.20330118]
 [4953.63050081]
 [4696.90555605]
 [5748.14728228]
 [3854.80419267]
 [4771.12638315]
 [3256.98200492]
 [5397.97576512]
 [4153.11368939]
 [4675.89088951]
 [4738.9484921 ]
 [4085.51299371]
 [3763.4167354 ]
 [3267.70133905]
 [4347.81654321]
 [4842.36992216]
 [4624.0459312 ]
 [3869.9682360

#### 03. csv 생성

In [20]:
pd.DataFrame({'body_mass_g': y_pred[:,0]}).to_csv('./yemoonsaBigdata/res/004000000.csv', index=False)